# Crime Datasets

In [ ]:
# Load  datasets
import numpy as np
import pandas as pd

crime2020 = pd.read_csv('crimes2020.csv')
crime2021 = pd.read_csv('crimes2021.csv')
crime2022 = pd.read_csv('crimes2022.csv')
crime2023 = pd.read_csv('crimes2023.csv')
crime2024 = pd.read_csv('crimes2024.csv')

**Feature Selection**

In [ ]:
# List of crime datasets
crime_datasets = [crime2020, crime2021, crime2022, crime2023, crime2024]

# Columns to keep in crime
columns_to_keep = ['Year', 'Yeshuv', 'StatisticGroup', 'StatisticGroupKod']

# Apply column selection
crime2020, crime2021, crime2022, crime2023, crime2024 = [
    df[columns_to_keep] for df in crime_datasets]

**Counting occurences of types of crime**

In [ ]:
# Function to group by 'Yeshuv' and 'StatisticGroup', counting occurrences of types of crime
def preprocess_dataset(df):
    grouped = df.groupby(['Yeshuv', 'StatisticGroup', 'StatisticGroupKod', 'Year']).size().reset_index(name='Count')
    return grouped

# Apply the function to each dataset
grouped_2020 = preprocess_dataset(crime2020)
grouped_2021 = preprocess_dataset(crime2021)
grouped_2022 = preprocess_dataset(crime2022)
grouped_2023 = preprocess_dataset(crime2023)
grouped_2024 = preprocess_dataset(crime2024)

# Combine all grouped datasets into one
combined_crime = pd.concat([grouped_2020, grouped_2021, grouped_2022, grouped_2023, grouped_2024], ignore_index=True)

**Remove error typing**

In [ ]:
# remove all rows where StatisticGroupKod is -1 (error typing)
combined_crime = combined_crime[combined_crime['StatisticGroupKod'] != -1].copy()
combined_crime.rename(columns={'Yeshuv': 'Settlement'}, inplace=True)

**Aggregate counts for all crimes together**

In [ ]:
# Aggregate counts of all crime types per Yeshuv and Year
total_crimes = combined_crime.groupby(['Settlement', 'Year'])['Count'].sum().reset_index()

# Add the new columns for total crime group
total_crimes['StatisticGroup'] = "כל העבירות"
total_crimes['StatisticGroupKod'] = -2

# Reorder columns to match combined_crime
total_crimes = total_crimes[['Settlement', 'StatisticGroup', 'StatisticGroupKod', 'Year', 'Count']]

# Append the total crimes rows to the combined dataframe
crime = pd.concat([combined_crime, total_crimes], ignore_index=True)

# Education dataset

In [ ]:
# Load datasets

education2019 = pd.read_excel('data2019.xlsx')
education2020 = pd.read_excel('data2020.xlsx')
education2021 = pd.read_excel('data2021.xlsx')
education2022 = pd.read_excel('data2022.xlsx')
education2023 = pd.read_excel('data2023.xlsx')

**Check features that not match in all education datasets**

In [ ]:
# identify which features (columns) are not common across all education datasets
# Store all column sets in a list
column_sets = [
    set(education2019.columns),
    set(education2020.columns),
    set(education2021.columns),
    set(education2022.columns),
    set(education2023.columns)
]

# Find the intersection (common columns in all datasets)
common_columns = set.intersection(*column_sets)

# Print out non-matching (unshared) features for each dataset
for i, (year, df) in enumerate(zip(range(2019, 2024), [education2019, education2020, education2021, education2021, education2023])):
    current_columns = set(df.columns)
    unmatched = current_columns.symmetric_difference(common_columns)
    if unmatched:
        print(f"Unmatched features in data{year}:")
        for col in sorted(unmatched):
            print(f"  - {col}")
    else:
        print(f"All features match in data{year}")
    print()

Unmatched features in data2019:
  - מספר תושבים ברשות לשנת תשעט

Unmatched features in data2020:
  - מספר תושבים ברשות לשנת תשף

Unmatched features in data2021:
  - מספר תושבים ברשות לשנת תשפא

Unmatched features in data2022:
  - מספר תושבים ברשות לשנת תשפא

Unmatched features in data2023:
  - מספר תושבים ברשות לשנת תשפג



**Rename not matched feature to common name in all datasets**

In [ ]:
# rename the column "מספר תושבים ברשות לשנת תשפג/תשפב/תשפא/תשף/תשעט" in each dataset to a common name "NumResidents"
# Define the mapping of old column names to the new one
resident_column_names = {
    "מספר תושבים ברשות לשנת תשעט": "NumResidents",
    "מספר תושבים ברשות לשנת תשף": "NumResidents",
    "מספר תושבים ברשות לשנת תשפא": "NumResidents",
    "מספר תושבים ברשות לשנת תשפב": "NumResidents",
    "מספר תושבים ברשות לשנת תשפג": "NumResidents",
}

# Apply renaming to each dataset
education2019.rename(columns=resident_column_names, inplace=True)
education2020.rename(columns=resident_column_names, inplace=True)
education2021.rename(columns=resident_column_names, inplace=True)
education2022.rename(columns=resident_column_names, inplace=True)
education2023.rename(columns=resident_column_names, inplace=True)

**Remove not relevant columns from education datasets**

In [ ]:
# Dictionary of datasets
datasets = {
    "education2019": education2019,
    "education2020": education2020,
    "education2021": education2021,
    "education2022": education2022,
    "education2023": education2023,
}

# List of columns to remove
columns_to_remove = [
    "מספר תלמידים  י'-יב'  חט\"ע - מצבת",
    "מספר תלמידי י\"ב לומדים ברשות  -מצבת",
    "שיעור תלמידים עולים",
    "אחוז תלמידי חינוך מיוחד הלומדים בחינוך רגיל",
    "אחוז תלמידים ברשות במוסדות חינוך מיוחד",
    "שיעור תלמידים בכיתה רגילה עם סל אישי (שילוב)",
    "שיעור תלמידים בכיתות חינוך מיוחד במוסדות רגילים",
    "כיתות לב עיוני",
    "אחוז נשירה מגרים",
    "שיעור מחברות פסולות",
    "אחוז זכאות לבגרות  מהגרים",
    "אחוז זכאות לבגרות  מהגרים חינוך רגיל",
    "אחוז זכאות לבגרות  מצטיינת מהגרים",
    "אחוז זכאות לבגרות  מצטיינת מהגרים חינוך רגיל",
    "גודל כיתה רשותי יסודי",
    "גודל כיתה רשותי חט\"ב",
    # "מספר תלמידים גרים ברשות   י-יב",
    "מספר תלמידי י\"ב ברשות חינוך רגיל",
    "מספר תלמידי י\"ב ברשות",
    "אחוז זכאים לבגרות טק ברשות",
    "אחוז זכאים לבגרות טק ברשות - חינוך רגיל",
    "אחוז זכאים לבגרות טק ברשות (ללא אנגלית)",
    "אחוז זכאים לבגרות טק ברשות (ללא אנגלית)  - חינוך רגיל",
    "אחוז זכאים לבגרות טק ארצי",
    "אחוז זכאים לבגרות טק ארצי (ללא אנגלית)",
    "מספר זכאים לבגרות טק חינוך רגיל",
    "מספר זכאים לבגרות טק",
    "מספר זכאים לבגרות טק (ללא אנגלית) חינוך רגיל",
    "מספר זכאים לבגרות טק (ללא אנגלית)",
        "סמל רשות",
    "שם מחוז גיאוגרפי",
    "תלמידי י\"ב במוסדות המגישים לבגרות",
    "שיעור תלמידים בכיתות חינוך מיוחד במוסדות רגילים "
]

# Drop the specified columns if they exist
for name, df in datasets.items():
    existing_cols = [col for col in columns_to_remove if col in df.columns]
    df.drop(columns=existing_cols, inplace=True)
    print(f"{name}: Removed {len(existing_cols)} columns.")

education2019: Removed 21 columns.
education2020: Removed 21 columns.
education2021: Removed 21 columns.
education2022: Removed 21 columns.
education2023: Removed 21 columns.


**Check null values in each town (row) in each dataset (year)**

In [ ]:
# Set the name of the column that contains town names
town_column = "שם רשות"  # Change this if your town column has a different name

# Function to analyze each dataset
def report_missing_per_town(df, dataset_name):
    print(f"\n--- {dataset_name} ---")
    for idx, row in df.iterrows():
        town_name = row.get(town_column, f"Row {idx}")
        null_count = row.isna().sum()
        dash_count = (row == "-").sum()
        total_issues = null_count + dash_count
        if total_issues > 0:
            print(f"{town_name}: {total_issues} missing values (NaN: {null_count}, '-': {dash_count})")

# Apply the function to each dataset
for name, df in datasets.items():
    report_missing_per_town(df, name)


--- education2019 ---
אזור: 20 missing values (NaN: 0, '-': 20)
אלונה: 20 missing values (NaN: 0, '-': 20)
אליכין: 20 missing values (NaN: 0, '-': 20)
אלפי מנשה: 20 missing values (NaN: 0, '-': 20)
בית אריה: 20 missing values (NaN: 0, '-': 20)
בית דגן: 20 missing values (NaN: 0, '-': 20)
בני עי"ש: 20 missing values (NaN: 0, '-': 20)
גבעת זאב: 6 missing values (NaN: 0, '-': 6)
גדרות: 20 missing values (NaN: 0, '-': 20)
הר-אדר: 20 missing values (NaN: 0, '-': 20)
חריש: 20 missing values (NaN: 0, '-': 20)
יסוד המעלה: 20 missing values (NaN: 0, '-': 20)
כוכב יאיר: 20 missing values (NaN: 0, '-': 20)
כפר ברא: 20 missing values (NaN: 0, '-': 20)
כפר קמא: 20 missing values (NaN: 0, '-': 20)
כפר שמריהו: 20 missing values (NaN: 0, '-': 20)
כפר תבור: 20 missing values (NaN: 0, '-': 20)
להבים: 20 missing values (NaN: 0, '-': 20)
לכיש: 20 missing values (NaN: 0, '-': 20)
מבואות החרמון: 6 missing values (NaN: 0, '-': 6)
מזרעה: 20 missing values (NaN: 0, '-': 20)
מטולה: 20 missing values (NaN: 0, '

**Remove some towns from datasets with a lot of null values**

Some towns in each dataset has bigger than 10 features (we have in each dataset 42 features) with value "-", we want to remove them. This means we remove 27-28 towns from each dataset where it was 255 towns (rows), it's mean we have 227-228 rows in each dataset after remove.

In [ ]:
# Set the maximum allowed "-" values per town
dash_threshold = 10

# Function to clean each dataset
def remove_towns_with_many_dashes(df, dataset_name):
    dash_counts = (df == "-").sum(axis=1)
    to_remove = dash_counts > dash_threshold
    num_removed = to_remove.sum()
    df_cleaned = df[~to_remove].copy()

    print(f"{dataset_name}: Removed {num_removed} towns with more than {dash_threshold} '-' values")
    return df_cleaned

# Clean each dataset
for name in datasets:
    datasets[name] = remove_towns_with_many_dashes(datasets[name], name)

education2019: Removed 27 towns with more than 10 '-' values
education2020: Removed 27 towns with more than 10 '-' values
education2021: Removed 29 towns with more than 10 '-' values
education2022: Removed 28 towns with more than 10 '-' values
education2023: Removed 27 towns with more than 10 '-' values


**Replace "-" with null value in all datasets**

In [ ]:
# Replace "-" with np.nan in all datasets
for name, df in datasets.items():
    datasets[name] = df.replace("-", np.nan)

<ipython-input-145-8f2dd14f9da5>:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  datasets[name] = df.replace("-", np.nan)


**Combine education datasets to one dataset**

In [ ]:
# Add a 'Year' column to each dataset before combining
for year in range(2019, 2024):
    name = f"education{year}"
    datasets[name]["Year"] = year

# Combine all datasets into a single DataFrame
education_all = pd.concat([datasets[f"education{year}"] for year in range(2019, 2024)], ignore_index=True)

# Preview the result
print("Combined dataset shape:", education_all.shape)
education_all.head()

Combined dataset shape: (1137, 37)


,"שנה""ל",שם רשות,אשכול סוציו-אקונומי,NumResidents,מספר תלמידים גרים ברשות,מספר תלמידים גרים ברשות י-יב,מספר בתי ספר,"מספר מוסדות שדווחו להם תלמידים כיתות י-י""ב",מספר תלמידים לומדים ברשות - מצבת,שיעור התלמידים בחינוך טכנולוגי,...,אחוז זכאות 4 יחידות אנגלית,אחוז זכאות 4 יחידות אנגלית -חינוך רגיל,אחוז זכאות 5 יחידות אנגלית,אחוז זכאות 5 יחידות אנגלית - חינוך רגיל,אחוז זכאות 4 יחידות מתמטיקה,אחוז זכאות 4 יחידות מתמטיקה- חינוך רגיל,אחוז זכאות 5 יחידות מתמטיקה,אחוז זכאות 5 יחידות מתמטיקה- חינוך רגיל,"גודל כיתה רשותי חט""ע",Year
0,תשעט,אבו גוש,3,8793,1674,396,5,3,1273,0.166205,...,0.158,0.182,0.208,0.239,0.149,0.170,0.040,0.045,24.166667,2019
1,תשעט,אבו סנאן,3,14235,2726,694,7,3,2506,0.175676,...,0.272,0.287,0.262,0.272,0.194,0.200,0.087,0.092,28.350000,2019
2,תשעט,אבן יהודה,8,15103,3180,667,7,3,3941,0.145274,...,0.184,0.186,0.592,0.589,0.216,0.213,0.263,0.265,25.880952,2019
3,תשעט,אום אל-פחם,2,59319,14111,3544,30,13,14368,0.455285,...,0.327,0.337,0.176,0.184,0.156,0.161,0.093,0.098,26.582734,2019
4,תשעט,אופקים,3,37837,6252,1388,26,10,6148,0.541122,...,0.183,0.189,0.171,0.183,0.109,0.118,0.043,0.047,23.428571,2019


**Remove and rename some features**

In [ ]:
# Remove the 'שנה"ל' column if it exists
if "שנה\"ל" in education_all.columns:
    education_all = education_all.drop(columns=["שנה\"ל"])
    print("Column 'שנה\"ל' removed.")

# Rename the column שם רשות
education_all = education_all.rename(columns={"שם רשות": "Settlement"})
print("Column 'שם רשות' renamed to 'Settlement'.")

Column 'שנה"ל' removed.
Column 'שם רשות' renamed to 'Settlement'.


**Feature Engineering**

In [ ]:
# feature engineering

education_all["HighSchoolStudentsPerHighClass"] = (
    education_all["מספר תלמידים גרים ברשות   י-יב"] / education_all["מספר כיתות י'-יב"])


education_all["TechClassRatio"] = (
    education_all["כיתות לב טכנולוגי"] / education_all["מספר כיתות י'-יב"])


education_all["GiftedClassRatio"] = (
    education_all["כיתות מחוננים"] / education_all["מספר כיתות י'-יב"])


education_all["StudentsPerResidentRatio"] = (
    education_all["מספר תלמידים גרים ברשות"] / education_all["NumResidents"])


education_all["HighSchoolCoverageRatio"] = (
    education_all["מספר מוסדות שדווחו להם תלמידים כיתות י-י\"ב"] / education_all["מספר בתי ספר"])


education_all["HilaStudentRatio"] = (
    education_all["תלמידי היל\"ה רשות לימודים"] / education_all["מספר תלמידים גרים ברשות"])


education_all["TeacherStudentRatio"] = (
    education_all["סך מורים"] / education_all["מספר תלמידים גרים ברשות"])


education_all["StudentPerClass"] = (
    education_all["מספר תלמידים לומדים ברשות - מצבת"] / education_all["מספר כיתות"])


education_all["StudentPerSchool"] = (
    education_all["מספר תלמידים לומדים ברשות - מצבת"] / education_all["מספר בתי ספר"])


education_all["OutstandingToTotal"] = (
    education_all["אחוז זכאות לבגרות  מצטיינת מהלומדים"] /
    education_all["אחוז זכאים לבגרות  מהלומדים"])


education_all["DropoutToPersistenceRatio"] = (
    education_all["אחוז נשירה מלומדים"] / education_all["אחוז התמדה"])


education_all["5YahidotIndex"] = (
    (education_all["אחוז זכאות 5 יחידות אנגלית"] +
     education_all["אחוז זכאות 5 יחידות מתמטיקה"]) / 2)


education_all["DropoutRiskScore"] = (
    education_all['תלמידי היל"ה רשות לימודים'] /
    education_all["מספר תלמידים לומדים ברשות - מצבת"]
) + education_all["אחוז נשירה מלומדים"]


education_all["AcademicExcellentScore"] = (
    education_all["אחוז זכאות 5 יחידות אנגלית"] +
    education_all["אחוז זכאות 5 יחידות מתמטיקה"] +
    education_all["אחוז זכאות לבגרות  מצטיינת מהלומדים"]) / 3


education_all["5YahidotIndexRegular"] = (
    education_all["אחוז זכאות 5 יחידות אנגלית - חינוך רגיל"] +
    education_all["אחוז זכאות 5 יחידות מתמטיקה-  חינוך רגיל"]) / 2


education_all["AcademicExcellentScoreRegular"] = (
    education_all["אחוז זכאות 5 יחידות אנגלית - חינוך רגיל"] +
    education_all["אחוז זכאות 5 יחידות מתמטיקה-  חינוך רגיל"] +
    education_all["אחוז זכאות לבגרות  מצטיינת-חינוך רגיל מהלומדים"]) / 3


education_all["4YahidotIndex"] = (
    education_all["אחוז זכאות 4 יחידות אנגלית"] +
    education_all["אחוז זכאות 4 יחידות מתמטיקה"]) / 2


education_all["4YahidotIndexRegular"] = (
    education_all["אחוז זכאות 4 יחידות אנגלית  -חינוך רגיל"] +
    education_all["אחוז זכאות 4 יחידות מתמטיקה-  חינוך רגיל"]) / 2

In [ ]:
# List of columns to remove
columns_to_remove = [
    "מספר תלמידים גרים ברשות",
    "מספר תלמידים גרים ברשות   י-יב",
    "מספר בתי ספר",
    'מספר מוסדות שדווחו להם תלמידים כיתות י-י"ב',
    "מספר תלמידים לומדים ברשות - מצבת",
    "מספר כיתות",
    "מספר כיתות י'-יב",
    "כיתות לב טכנולוגי",
    "כיתות מחוננים",
    "סך מורים",
    'תלמידי היל"ה רשות לימודים'
]

# Drop the columns
education_all.drop(columns=columns_to_remove, inplace=True)

In [ ]:
# Translate all features to english
# Dictionary for renaming Hebrew columns to English
rename_dict = {
    'אשכול סוציו-אקונומי': 'SocioeconomicCluster',
    'שיעור התלמידים בחינוך טכנולוגי': 'PercentTechStudents',
    'שיעור המורים בעלי תואר שני ומעלה': 'PercentTeachersMasters',
    'שיעור המורים בעלי תואר שני ומעלה  חט"ע': 'PercentHighSchoolTeachersMasters',
    'אחוז התאמות ועדה מחוזית': 'PercentRegionalAccommodations',
    'אחוז התאמות מוסדי': 'PercentInstitutionalAccommodations',
    'אחוז התמדה': 'PersistenceRate',
    'אחוז נשירה מלומדים': 'DropoutRate',
    'אחוז שאלוני שק"ד': 'PercentShakdQuestionnaires',
    'אחוז זכאים לבגרות  מהלומדים': 'PercentBagrutEligible',
    'אחוז זכאים לבגרות - חינוך רגיל מהלומדים': 'PercentBagrutEligibleRegular',
    'אחוז זכאות לבגרות  מצטיינת מהלומדים': 'PercentExcellentBagrut',
    'אחוז זכאות לבגרות  מצטיינת-חינוך רגיל מהלומדים': 'PercentOutstandingBagrutRegular',
    'אחוז זכאות 4 יחידות אנגלית': 'Percent4UnitsEnglish',
    'אחוז זכאות 4 יחידות אנגלית  -חינוך רגיל': 'Percent4UnitsEnglishRegular',
    'אחוז זכאות 5 יחידות אנגלית': 'Percent5UnitsEnglish',
    'אחוז זכאות 5 יחידות אנגלית - חינוך רגיל': 'Percent5UnitsEnglishRegular',
    'אחוז זכאות 4 יחידות מתמטיקה': 'Percent4UnitsMath',
    'אחוז זכאות 4 יחידות מתמטיקה-  חינוך רגיל': 'Percent4UnitsMathRegular',
    'אחוז זכאות 5 יחידות מתמטיקה': 'Percent5UnitsMath',
    'אחוז זכאות 5 יחידות מתמטיקה-  חינוך רגיל': 'Percent5UnitsMathRegular',
    'גודל כיתה רשותי חט"ע': 'AvgClassSizeHighSchool',
    '5YahidotIndex':'5UnitsIndex',
    '5YahidotIndexRegular':'5UnitsIndexRegular',
    '4YahidotIndex':'4UnitsIndex',
    '4YahidotIndexRegular':'4UnitsIndexRegular',
    'AcademicExcellentScore':'SchoolExcellentScore',
    'AcademicExcellentScoreRegular':'SchoolExcellentScoreRegular',
    }

# Rename the columns
education_all.rename(columns=rename_dict, inplace=True)

**All new features of education after feature engineering and selection:**
-	Settlement
-	SocioeconomicCluster – אשכול סוציו-אקונומי
-	NumResidents – מספר תושבים בעיר
-	PercentTechStudents - שיעור התלמידים בחינוך טכנולוגי
-	PercentTeachersMasters - שיעור המורים בעלי תואר שני ומעלה
-	PercentHighSchoolTeachersMasters - שיעור המורים בעלי תואר שני ומעלה  חט"ע
-	PercentRegionalAccommodations - אחוז התאמות ועדה מחוזית
-	PercentInstitutionalAccommodations - אחוז התאמות מוסדי
-	PersistenceRate - אחוז התמדה
-	DropoutRate - אחוז נשירה מלומדים
-	PercentShakdQuestionnaires - אחוז שאלוני שק"ד, שאלונים שיכולים לבחור אם לכלול בחישוב ממוצע הבגרות
-	PercentBagrutEligible - אחוז זכאים לבגרות  מהלומדים
-	PercentBagrutEligibleRegular - אחוז זכאים לבגרות - חינוך רגיל מהלומדים
-	PercentExcellentBagrut - אחוז זכאות לבגרות  מצטיינת מהלומדים
-	PercentOutstandingBagrutRegular - אחוז זכאות לבגרות  מצטיינת-חינוך רגיל מהלומדים
-	Percent4UnitsEnglish - אחוז זכאות 4 יחידות אנגלית
-	Percent4UnitsEnglishRegular - אחוז זכאות 4 יחידות אנגלית  -חינוך רגיל
-	Percent5UnitsEnglish - אחוז זכאות 5 יחידות אנגלית
-	Percent5UnitsEnglishRegular - אחוז זכאות 5 יחידות אנגלית - חינוך רגיל
-	Percent4UnitsMath - אחוז זכאות 4 יחידות מתמטיקה
-	Percent4UnitsMathRegular - אחוז זכאות 4 יחידות מתמטיקה-  חינוך רגיל
-	Percent5UnitsMath - אחוז זכאות 5 יחידות מתמטיקה
-	Percent5UnitsMathRegular - אחוז זכאות 5 יחידות מתמטיקה-  חינוך רגיל
-	AvgClassSizeHighSchool - גודל כיתה רשותי חט"ע
-	Year
-	HighSchoolStudentsPerHighClass - ממוצע תלמידים בכיתות י'-י"ב (מדד לצפיפות בכיתות תיכון)
-	TechClassRatio - יחס כיתות טכנולוגיות מתוך כלל כיתות י'-י"ב (מדד להתמקדות בחינוך טכנולוגי)
-	GiftedClassRatio - יחס כיתות מחוננים מתוך כלל כיתות י'-י"ב (מדד לתלמידים מצטיינים)
-	StudentsPerResidentRatio - שיעור התלמידים מתוך כלל התושבים (מדד לגיל האוכלוסייה/משקל מערכת החינוך)
-	HighSchoolCoverageRatio - שיעור בתי הספר בהם יש תלמידי י'-י"ב (כיסוי חינוכי לתיכונים)
-	HilaStudentRatio - שיעור תלמידי היל"ה מתוך כלל תלמידי הרשות (מדד לאוכלוסיות בסיכון/נוער מנותק)
-	TeacherStudentRatio - יחס מורים לתלמידים ברשות (מדד לעומס על מורים או איכות ההוראה)
-	StudentPerClass - מספר תלמידים ממוצע בכיתה (מדד לצפיפות כיתתית)
-	StudentPerSchool - ממוצע תלמידים לבית ספר (מדד לגודל מוסדות הלימוד)
-	OutstandingToTotal - שיעור זכאות לבגרות מצטיינת מתוך כלל הזכאים (מדד איכותי בתוך קבוצת ההצלחה)
-	DropoutToPersistenceRatio - יחס בין נשירה להתמדה (מדד לשמירה על תלמידים במערכת)
-	5UnitsIndex - ממוצע שיעור הזכאות ל-5 יח"ל אנגלית ומתמטיקה (מדד למצוינות אקדמית)
-	DropoutRiskScore - מדד מורכב לסיכון לנשירה, כולל תלמידי היל"ה ואחוז נשירה
-	SchoolExcellentScore - ממוצע של זכאות ל-5 יח"ל אנגלית, מתמטיקה ובגרות מצטיינת (מדד למצוינות כוללת)
-	5UnitsIndexRegular - ממוצע זכאות ל-5 יח"ל אנגלית ומתמטיקה בחינוך הרגיל בלבד
-	SchoolExcellentScoreRegular ממוצע של זכאות ל-5 יח"ל אנגלית, מתמטיקה ובגרות מצטיינת בחינוך רגיל (מדד למצוינות כוללת)
-	4UnitsIndex - ממוצע זכאות ל-4 יח"ל אנגלית ומתמטיקה (מדד הישגים בינוניים-גבוהים)
-	4UnitsIndexRegular - ממוצע זכאות ל-4 יח"ל אנגלית ומתמטיקה בחינוך רגיל (מדד הישגים בינוניים-גבוהים)

During feature engineering, we ensured that no data leakage occurred. Specifically, we did not include any information from future time points relative to the target prediction year. Additionally, we avoided using the target variable or any features directly derived from it as inputs. All engineered features were based solely on historical and current data available at the time of prediction, maintaining a clean and realistic modeling framework.

# Combine crime and education datasets

In [ ]:
crime = pd.read_csv("crime.csv")

**Check settlements for matching**

In [ ]:
# Get unique settlements from each dataset
education_settlements = set(education_all["Settlement"].dropna().unique())
crime_settlements = set(crime["Settlement"].dropna().unique())

# Settlements in education but not in crime
only_in_education = education_settlements - crime_settlements

# Settlements in crime but not in education
only_in_crime = crime_settlements - education_settlements

# Print results
print(f"Settlements only in education dataset: {len(only_in_education)}")
print(sorted(only_in_education))

print(f"\nSettlements only in crime dataset: {len(only_in_crime)}")
print(sorted(only_in_crime))

Settlements only in education dataset: 83
['אל בטוף', 'אל קסום', 'אשכול', 'באר טוביה', 'בוסתאן אל מרג', "בועיינה נוג'ידאת", 'בוקעאתה', 'ביר אל-מכסור', 'בני שמעון', 'בנימינה גבעת עדה', 'בסמ"ה', 'בענה', 'ברנר', "ג'סר א-זרקא", 'גבעת זאב', 'גולן', 'גוש חלב', 'גוש עציון', 'גזר', 'גן רווה', 'דאלית אל-כרמל', 'דייר אל-אסד', 'דיר חנא', 'דרום השרון', 'הגלבוע', 'הגליל העליון', 'הגליל התחתון', 'הערבה התיכונה', 'הר חברון', 'זבולון', 'חבל אילות', 'חבל יבנה', 'חבל מודיעין', 'חוף אשקלון', 'חוף הכרמל', 'חוף השרון', 'טובא-זנגריה', 'יהוד נווה אפרים', 'יואב', "ינוח-ג'ת", "כאוכב אבו אל-היג'א", "כעביה-טבאש-חג'אג'רה", 'לב השרון', 'לכיש', 'מבואות החרמון', "מג'אר", "מג'ד אל-כרום", 'מגידו', 'מגילות ים המלח', 'מודיעין מכבים רעות', 'מטה אשר', 'מטה בנימין', 'מטה יהודה', 'מנשה', 'מעלה יוסף', 'מקווה ישראל', 'מרום הגליל', 'מרחבים', 'משגב', 'נווה מדבר', 'נחל שורק', "עג'ר", 'עילבון', "עין מאה'ל", 'עמק הירדן', 'עמק המעיינות', 'עמק חפר', 'עמק יזרעאל', 'ערבות הירדן', 'ערערה בנגב', 'פקיעין - בוקייעה', 'פתח תקוה', 'צורן קדי

Detect typos or formatting differences in town names that refer to the same place but are written differently in the education and crime datasets. I find 23 differences in town names.

In [ ]:
# Example mapping (you can expand it)
settlement_mapping = {
    'בוקעתא': 'בוקעאתה',
    'ביענה': 'בענה',
    'ביר אל מכסור': 'ביר אל-מכסור',
    'בנימינה-גבעת עדה': 'בנימינה גבעת עדה',
    "בסמ''ה": 'בסמ"ה',
    "ג'סר א זרקא": "ג'סר א-זרקא",
    'דאלית אל כרמל': 'דאלית אל-כרמל',
    'דייר אל אסד': 'דייר אל-אסד',
    'דייר חנא': 'דיר חנא',
    'טובא זנגריה': 'טובא-זנגריה',
    "יאנוח-ג'ת": "ינוח-ג'ת",
    'מגאר': "מג'אר",
    "מג'ד אל כרום": "מג'ד אל-כרום",
    'מודיעין-מכבים-רעות': 'מודיעין מכבים רעות',
    "ע'ג'ר": "עג'ר",
    'עיילבון': 'עילבון',
    'עין מאהל': "עין מאה'ל",
    'ערערה-בנגב': 'ערערה בנגב',
    'פקיעין': 'פקיעין - בוקייעה',
    'פתח תקווה': 'פתח תקוה',
    'שבלי': 'שבלי-אום אל-גנם',
    'שגב-שלום': 'שגב שלום',
    'תל אביב יפו': 'תל אביב-יפו'
}

# Apply mapping to crime dataset
crime['Settlement'] = crime['Settlement'].replace(settlement_mapping)

**Remove not matching towns in both datasets**

In [ ]:
# Remove not matching towns in both datasets
# Get matching settlements in both datasets
matching_settlements = set(education_all["Settlement"]).intersection(set(crime["Settlement"]))

# Filter both datasets to keep only matching settlements
education_all = education_all[education_all["Settlement"].isin(matching_settlements)].reset_index(drop=True)
crime = crime[crime["Settlement"].isin(matching_settlements)].reset_index(drop=True)

**Check for null values**

In [ ]:
# After removing not matched towns from datasets, we haven't null values in education dataser
for name, df in education_all.items():
    total_nans = df.isna().sum().sum()
    print(f"{name}: Total NaN values = {total_nans}")

Settlement: Total NaN values = 0
SocioeconomicCluster: Total NaN values = 0
NumResidents: Total NaN values = 0
PercentTechStudents: Total NaN values = 0
PercentTeachersMasters: Total NaN values = 0
PercentHighSchoolTeachersMasters: Total NaN values = 0
PercentRegionalAccommodations: Total NaN values = 0
PercentInstitutionalAccommodations: Total NaN values = 0
PersistenceRate: Total NaN values = 0
DropoutRate: Total NaN values = 0
PercentShakdQuestionnaires: Total NaN values = 0
PercentBagrutEligible: Total NaN values = 0
PercentBagrutEligibleRegular: Total NaN values = 0
PercentExcellentBagrut: Total NaN values = 0
PercentOutstandingBagrutRegular: Total NaN values = 0
Percent4UnitsEnglish: Total NaN values = 0
Percent4UnitsEnglishRegular: Total NaN values = 0
Percent5UnitsEnglish: Total NaN values = 0
Percent5UnitsEnglishRegular: Total NaN values = 0
Percent4UnitsMath: Total NaN values = 0
Percent4UnitsMathRegular: Total NaN values = 0
Percent5UnitsMath: Total NaN values = 0
Percent5Un

**Create merged dataset where match Education year t with Crime year t+1**

In [ ]:
# Create merged dataset where match Education year t with Crime year t+1
# Filter only total crime ("כל העבירות") BEFORE merging
crime_filtered = crime[crime['StatisticGroup'] == 'כל העבירות'].copy()

# Make copies to avoid modifying originals
education_lag1 = education_all.copy()
crime_lag1 = crime_filtered.copy()


# Shift the crime data backwards by 1 year for Lag 1
crime_lag1['Year'] = crime_lag1['Year'] - 1

# Merge on 'Settlement' and 'Year'
merged_lag1 = pd.merge(
    education_lag1,
    crime_lag1,
    on=['Settlement', 'Year'],
    how='inner',  # Use inner to only keep matched years
    suffixes=('_edu', '_crime')
)

columns_to_drop = [
    'StatisticGroup',
    'StatisticGroupKod'
]

# Drop columns from merged_lag1
final_data = merged_lag1.drop(columns=columns_to_drop, errors='ignore')

final_data['CrimeRate'] = final_data['Count'] / final_data['NumResidents']

final_data = final_data.drop(columns=['Count', 'NumResidents'])

In [ ]:
final_data

,Settlement,SocioeconomicCluster,PercentTechStudents,PercentTeachersMasters,PercentHighSchoolTeachersMasters,PercentRegionalAccommodations,PercentInstitutionalAccommodations,PersistenceRate,DropoutRate,PercentShakdQuestionnaires,...,OutstandingToTotal,DropoutToPersistenceRatio,5UnitsIndex,DropoutRiskScore,SchoolExcellentScore,5UnitsIndexRegular,SchoolExcellentScoreRegular,4UnitsIndex,4UnitsIndexRegular,CrimeRate
0,אבו גוש,3,0.166205,0.371429,0.394737,0.162791,0.453488,0.9296,0.006211,0.031746,...,0.047319,0.006682,0.1240,0.024279,0.092667,0.1420,0.106000,0.1535,0.1760,0.051291
1,אבו סנאן,3,0.175676,0.324138,0.321429,0.194872,0.035897,0.9079,0.017279,0.070423,...,0.163359,0.019031,0.1745,0.018875,0.152000,0.1820,0.159000,0.2330,0.2435,0.035757
2,אבן יהודה,8,0.145274,0.461538,0.532374,0.562992,0.192913,0.8630,0.015892,0.000000,...,0.172686,0.018415,0.4275,0.017415,0.336000,0.4270,0.336000,0.2000,0.1995,0.022843
3,אום אל-פחם,2,0.455285,0.335059,0.438503,0.170511,0.243822,0.9539,0.006898,0.016701,...,0.171200,0.007231,0.1345,0.012257,0.125333,0.1410,0.131667,0.2415,0.2490,0.032351
4,אופקים,3,0.541122,0.301775,0.295238,0.148148,0.175926,0.7723,0.011957,0.007519,...,0.035639,0.015483,0.1070,0.015536,0.077000,0.1150,0.083000,0.1460,0.1535,0.035759
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
841,שעב,2,0.000000,0.690789,0.743590,0.077670,0.388350,0.9474,0.004630,0.017544,...,0.145161,0.004887,0.2525,0.011531,0.207333,0.2600,0.213333,0.2425,0.2450,0.048869
842,שפרעם,3,0.584416,0.591787,0.560976,0.042647,0.407353,0.9108,0.006446,0.036717,...,0.078541,0.007077,0.1650,0.007294,0.128667,0.1710,0.133333,0.2265,0.2320,0.045266
843,תל אביב-יפו,8,0.370188,0.440750,0.546207,0.081457,0.513787,0.8988,0.012113,0.004638,...,0.176545,0.013476,0.4060,0.014545,0.317333,0.4395,0.344333,0.1815,0.1950,0.078783
844,תל מונד,9,0.166895,0.540146,0.631579,0.099548,0.846154,0.9250,0.006494,0.010638,...,0.362222,0.007020,0.5790,0.008012,0.494667,0.5995,0.511667,0.1900,0.1945,0.022449
